<a href="https://colab.research.google.com/github/Manos-Mak120/-/blob/Articles-Scrapper/BIDEN_ARTICLES_ATHENS_VOICE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import time
import random

from tqdm import tqdm

In [ ]:
def get_webpage_soup(url):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    return None
  else:
    # print(f"{url} was scraped successfully")
    return BeautifulSoup(response.text, 'html.parser')

In [ ]:
def get_article_elements_from_soup(soup, article_url):
  # απομόνωση του block με το περιεχόμενο του άρθρου από όλη τη σούπα της σελίδας
  article_block = soup.find("div", {"class": "articleSingle articleSingle--default"})
  # article url
  article_url = article_url
  # title
  try:
    title = article_block.find("h1", {"class": "articleSingle__title"}).text.strip()
  except:
    # try:
    #   title = soup.find("div", {"class": "container is-fullhd p-3"}).h1.text.strip() # προσθήκη για να πιάνει και τον τίτλο όταν βρίσκεται σε αυτό το div
    # except:
    title=""
  # datetime
  try:
    datetime_str = article_block.find("time")["datetime"]
  except:
    try:
      datetime_str = soup.find("li", {"class": "actionsList__item actionsList__item--date"}).find("time")["datetime"]
    except:
      datetime_str = ""
      date_str = ""
      time_str = ""
  if datetime_str != "":
    date_str = datetime_str.split("T")[0]
    time_str = datetime_str.split("T")[1].split("Z")[0]
  # section
  try:
    section = article_block.find("a", class_=lambda c: c and "articleSingle__category" in c).text.strip()
  except:
    section = ""
  # author
  try:
    author = article_block.find("a", class_=lambda c: c and "actionsList__link" in c).find("span").text.strip()
  except:
    # try:
    #   author = soup.find("div", {"class": "container is-fullhd p-3"}).find("a", {"rel": "author"}).text.strip() # προσθήκη
    # except:
    author = ""
  #subtitle
  try:
    subtitle = article_block.find("div", {"class": "articleSingle__intro"}).text.strip()
  except:
    subtitle = ""
  # excerpt
  try:
    excerpt = article_block.find("div", {"class": "articleSingle__summaryContent"}).find("p").text.strip()
  except:
    # try:
    #   excerpt = soup.find("div", {"class": "container is-fullhd p-3"}).find("div", {"class": "newsletter-description nx-excerpt my-5"}).text.strip() # προσθήκη
    # except:
    excerpt=""
  # text body
  try:
    text_body_p_tagsL = article_block.find("div", class_="articleSingle__description").find_all("p")
    text_body_textsL = []
    for p_tag in text_body_p_tagsL:
      p_tag_text = p_tag.text.strip()
      text_body_textsL.append(p_tag_text)
    text_body = " ".join(text_body_textsL)
  except:
    text_body=""
  # feature image
  try:
    feat_img_src = article_block.find("div", class_="articleSingle__image").find('img')["src"]
    feat_img_link = "https://www.athensvoice.gr"+feat_img_src
  except:
    feat_img_link=""


  # Δημιουργία λεξικού με τα data του άρθρου
  articleD = {"article_url": article_url, "title": title, "date": date_str, "time": time_str, "section": section, "author": author,
              "excerpt": excerpt, "subtitle": subtitle, "text_body": text_body, "feat_img_link": feat_img_link}

  return articleD

In [ ]:
def scrape_athensvoice_articles_pages(teasers_df):
  all_articles_dataL = []
  for idx, row in tqdm(teasers_df.iterrows(), total=teasers_df.shape[0], desc="Processing rows"):

    article_url = row['article_url']
    soup = get_webpage_soup(article_url)
    if soup is not None:
      articleD = get_article_elements_from_soup(soup, article_url)
      all_articles_dataL.append(articleD)
    else:
      # Optional: Print or log the URL that failed to scrape
      print(f"Warning: Could not scrape {article_url}")
    # articleD = get_article_elements_from_soup(soup, article_url)
    # all_articles_dataL.append(articleD)
    delay = random.uniform(1, 3)
    time.sleep(delay)
  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetched {len(df)} articles.")
  return df

**Δοκιμή του scraper**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
biden_21 = pd.read_csv("/content/drive/MyDrive/Athens.voice/Scrapper/biden_df_2020_2021.csv")

In [ ]:
articles_df = scrape_athensvoice_articles_pages(biden_21)
articles_df

Processing rows: 100%|██████████| 556/556 [31:40<00:00,  3.42s/it]


Scraping completed. Fetched 556 articles.


,article_url,title,date,time,section,author,excerpt,subtitle,text_body,feat_img_link
0,https://www.athensvoice.gr/epikairotita/diethn...,Μπάιντεν σε Πούτιν: Θα αντιδράσουμε σε εισβολή...,2021-12-31,05:58:00,Κοσμος,Newsroom,Η Ουάσινγκτον θα αντιδράσει «αποφασιστικά» σε ...,Η απάντηση του Βλαντιμίρ Πούτιν,Την πρόθεσή των ΗΠΑ να αντιδράσουν αποφασιστικ...,https://www.athensvoice.gr/images/1074x600/3/j...
1,https://www.athensvoice.gr/epikairotita/diethn...,Σήμερα η τηλεφωνική επικοινωνία Τζο Μπάιντεν &...,2021-12-30,06:36:06,Κοσμος,Newsroom,Ο πρόεδρος των ΗΠΑ Τζο Μπάιντεν και ο Ρώσος ομ...,Ποια θέματα βρίσκονται στην ατζέντα των συζητή...,Τηλεφωνική επικοινωνία θα έχουν σήμερα ο πρόεδ...,https://www.athensvoice.gr/images/1074x600/3/j...
2,https://www.athensvoice.gr/epikairotita/diethn...,Πούτιν και Μπάιντεν θα έχουν ξανά επικοινωνία ...,2021-12-29,20:15:00,Κοσμος,Newsroom,Ο πρόεδρος των ΗΠΑ Τζο Μπάιντεν και ο Ρώσος ομ...,Εν μέσω ανησυχιών για ρωσική εισβολή,Ο πρόεδρος των ΗΠΑ Τζο Μπάιντεν και ο Ρώσος ομ...,https://www.athensvoice.gr/images/1074x600/3/j...
3,https://www.athensvoice.gr/epikairotita/athlit...,Τζον Μάντεν: Πέθανε ο θρύλος του NFL,2021-12-29,12:39:00,Αθλητισμος,Newsroom,Τζον Μάντεν: Έφυγε από τη ζωή σε ηλικία 85 ετώ...,Σε ηλικία 85 ετών,Την τελευταία του πνοή άφησε τα ξημερώματα στι...,https://www.athensvoice.gr/images/1074x600/3/j...
4,https://www.athensvoice.gr/epikairotita/diethn...,Πολίτης εξύβρισε τον Μπάιντεν κατά τη διάρκεια...,2021-12-25,07:13:08,Κοσμος,Newsroom,ΗΠΑ: Πολίτης εξύβρισε τον Μπάιντεν κατά τη διά...,Τι είναι η φράση: «Let's go Brandon»,"Ο αμερικανός πρόεδρος, Τζο Μπάιντεν, υπέστη μά...",https://www.athensvoice.gr/images/1074x600/3/j...
...,...,...,...,...,...,...,...,...,...,...
551,https://www.athensvoice.gr/epikairotita/diethn...,Μπάιντεν για Αγία Σοφία: Τι αποκάλυψε ο αρχιεπ...,2020-07-17,15:25:00,Κοσμος,Newsroom,Ο Τζο Μπάιντεν τηλεφώνησε στον Αρχιεπίσκοπο Αμ...,Το μήνυμα αλληλεγγύης που έστειλε ο Δημοκρατικ...,Τηλεφωνική επικοινωνία για την Αγία Σοφία με τ...,https://www.athensvoice.gr/images/1074x600/3/j...
552,https://www.athensvoice.gr/epikairotita/diethn...,Μπάιντεν: Ποιοι υπέγραψαν εξαψήφιες επιταγές γ...,2020-07-16,18:23:00,Κοσμος,Newsroom,"Ο Μπάιντεν ξεπέρασε τον Τραμπ, για πρώτη φορά,...",Τα έσοδα για την προεκλογική εκστρατεία του υπ...,Εύποροι δωρητές έδωσαν προβάδισμα στον Δημοκρα...,https://www.athensvoice.gr/images/1074x600/3/j...
553,https://www.athensvoice.gr/epikairotita/diethn...,Αγία Σοφία - Μπάιντεν: Καλώ τον Ερντογάν να αν...,2020-07-11,14:19:00,Κοσμος,Newsroom,Ο Τζο Μπάιντεν κάλεσε τον Ερντογάν να αναιρέσ...,«Λυπάμαι για την απόφαση της τουρκικής κυβέρνη...,Να αναιρέσει την απόφασή του για τη μετατροπή ...,https://www.athensvoice.gr/images/1074x600/3/j...
554,https://www.athensvoice.gr/epikairotita/politi...,Σε τι πανεπιστήμιο μπαίνουν σήμερα τα παιδιά,2020-07-10,06:48:00,Πολιτικη & Οικονομια,Χρύσα Μακρή,Εισαγωγή στα πανεπιστήμια: Τι αλλάζει από φέτο...,"Φέτος, για πρώτη φορά, θα εφαρμοστεί εκτός από...",Οι καιροί αλλάζουν και το μέλλον μιας χώρας κρ...,https://www.athensvoice.gr/images/1074x600/3/j...


In [ ]:
articles_df['text_body'][0]  #έλεγχος ότι πήρε όλο το κείμενο

'Την πρόθεσή των ΗΠΑ να αντιδράσουν αποφασιστικά σε περίπτωση περαιτέρω εισβολής της Ρωσίας στην Ουκρανία γνωστοποίησε ο Τζο Μπάιντεν στον Βλαντιμίρ Πούτιν κατά την τηλεφωνική τους επικοινωνία χθες, Πέμπτη 30 Δεκεμβρίου 2021. «Ο πρόεδρος Μπάιντεν κάλεσε τη Ρωσία να αποκλιμακώσει τις εντάσεις με την Ουκρανία», ανέφερε η εκπρόσωπος της αμερικανικής προεδρίας, η Τζεν Ψάκι, σε ανακοίνωση Τύπου που δημοσιοποίησε. «Κατέστησε σαφές ότι οι ΗΠΑ και οι εταίροι τους θα αντιδράσουν αποφασιστικά εάν η Ρωσία εισβάλει περαιτέρω στην Ουκρανία», πρόσθεσε. Η οποιαδήποτε διπλωματική πρόοδος στις αμερικανορωσικές σχέσεις περνάει από την «αποκλιμάκωση» στην Ουκρανία, επέμεινε η κυρία Ψάκι. Η επιβολή κυρώσεων από τις ΗΠΑ και τους συμμάχους τους στη Ρωσία εξαιτίας της κρίσης στην Ουκρανία θα αποτελούσε «κολοσσιαίο λάθος» και θα μπορούσε να οδηγήσει ακόμη και στη διακοπή των σχέσεων ανάμεσά τους, προειδοποίησε ο Ρώσος πρόεδρος Βλαντίμιρ Πούτιν τον Αμερικανό ομόλογό του Τζο Μπάιντεν, σύμφωνα με το Κρεμλίνο. Η 

In [ ]:
articles_df['feat_img_link'][0]

'https://www.athensvoice.gr/images/1074x600/3/jpg/sites/default/files/article/2021/12/30/biden-putin.jpg'

In [ ]:
from datetime import datetime

start_date = datetime(2020, 11, 20)
end_date = datetime(2021, 4, 30)

articles_df['date'] = pd.to_datetime(articles_df['date'], errors='coerce')

filtered_articles_df = articles_df[(articles_df['date'] >= start_date) & (articles_df['date'] <= end_date)].copy()

print(f"\nFiltered articles from 20/11/2020 to April 2021: {len(filtered_articles_df)}")


Filtered articles from 20/11/2020 to April 2021: 216


In [ ]:
duplicates_url = filtered_articles_df[filtered_articles_df.duplicated(subset=['article_url'], keep=False)]
print(f"\nArticles with duplicate URLs: {len(duplicates_url)}")
if not duplicates_url.empty:
    print(duplicates_url[['article_url', 'title']].sort_values(by='article_url'))

# Check for articles without title
no_title = filtered_articles_df[filtered_articles_df['title'].isnull() | (filtered_articles_df['title'].str.strip() == '')]
print(f"\nArticles without a title: {len(no_title)}")
if not no_title.empty:
    print(no_title[['article_url', 'title']])

# Check for articles without text body
no_text = filtered_articles_df[filtered_articles_df['text_body'].isnull() | (filtered_articles_df['text_body'].str.strip() == '')]
print(f"\nArticles without text body: {len(no_text)}")
if not no_text.empty:
    print(no_text[['article_url', 'title']])


Articles with duplicate URLs: 0

Articles without a title: 1
                                           article_url title
299  https://www.athensvoice.gr/epikairotita/diethn...      

Articles without text body: 9
                                           article_url  \
166  https://look.athensvoice.gr/look/celebrities/7...   
189  https://look.athensvoice.gr/look/celebrities/7...   
236  https://www.athensvoice.gr/advertorial/market/...   
245  https://look.athensvoice.gr/look/lifestyle/700...   
248  https://www.athensvoice.gr/advertorial/700705/...   
265  https://look.athensvoice.gr/look/fashion/69901...   
275  https://look.athensvoice.gr/look/lifestyle/698...   
299  https://www.athensvoice.gr/epikairotita/diethn...   
318  https://look.athensvoice.gr/look/celebrities/6...   

                                                 title  
166  Κιμ Καρντάσιαν - Γενοκτονία Αρμενίων: Ευχαριστ...  
189  Τζιλ Μπάιντεν: Το διχτυωτό καλσόν άναψε «φωτιέ...  
236               ABSOLUTMURAL.GR

In [ ]:
biden_articles_20_21= filtered_articles_df

In [ ]:
# Drop specific rows by index
biden_articles_20_21 = biden_articles_20_21.drop([166,189,236,245,248,265,275,299,318,299])

In [ ]:
from google.colab import drive

In [ ]:
biden_articles_20_21.to_csv('/content/drive/MyDrive/Athens.voice/Articles/biden_articles_df_2020_2021.csv', index=False)